## Introduction
<div style="text-align: justify;">
In this report, we will explore the functionalities of CoordChemPy, a Python package capable of modelizing monometallic coordination complexes from the TMQM database. This programme posses the functionality of identifying the central atome of very complex coordination compounds as well as the number of ligands to which it is bonded. In addition, CoordChemPy  is able to calulate the angles between bonds and estimate, if the bonds angles are relevant, the geometry of the coordination compound. In this short report, the purpose and usage of each function in the code will be discussed. Moreover, examples illustrating how the functions work will be provided. Finally, the difficulties encounterd during the project will be discussed. 

## Code Overview
The program is structured into several functions, each responsible for specific tasks in the analysis and visualization of coordination compounds. The key functions include:

In [ ]:
1. calculate_distance
2. get_covalent_radius
3. read_lines_around_keyword
4. read_xyz
5. infer_bonds
6. find_central_atom
7. find_ligands
8. find_actual_ligand_count
9. calculate_angle
10. determine_geometry
11. calculate_angles_and_geometry
12. cn
13. charge
14. visualize_label
15. visualize
16. visualize_all_data

## Functionality Details
**1. calculate_distance**

Calculates the Euclidean distance between two 3D coordinates.

In [ ]:
def calculate_distance(coord1, coord2):
    """
    Calculate the Euclidean distance between two 3D coordinates.

    Args:
        coord1 (tuple): The coordinates of the first point.
        coord2 (tuple): The coordinates of the second point.

    Returns:
        float: The Euclidean distance between the two points.
    """
    return np.linalg.norm(np.array(coord1) - np.array(coord2))

**Example:**

In [ ]:
import coordchempy
from coordchempy import calculate_distance
distance = calculate_distance((0, 0, 0), (1, 1, 1))
print(distance)  # Output: 1.7320508075688772

**2. get_covalent_radius**

Retrieves the covalent radius of an atom using the Mendeleev package.

In [ ]:
def get_covalent_radius(atom_symbol):
    """
    Get the covalent radius of an atom in angstroms using the Mendeleev package.

    Args:
        atom_symbol (str): The symbol of the atom.

    Returns:
        float: The covalent radius of the atom in angstroms.
    """
    radius_pm = element(atom_symbol).covalent_radius
    return radius_pm / 100.0  # Convert pm to Å

**Example:**

In [ ]:
import coordchempy
from coordchempy import get_covalent_radius
radius = get_covalent_radius('C')
print(radius)  # Output: 0.75 (depends on the element data in the Mendeleev package)

**3. read_lines_around_keyword**

Reads lines around a keyword from specified files

In [ ]:
def read_lines_around_keyword(keyword, filenames=None, lines_before=1):
    """
    Read lines around a keyword from specified filenames.

    Args:
        keyword (str): The keyword to search for.
        filenames (list, optional): List of filenames to search in. Defaults to ['tmQM_X1.xyz', 'tmQM_X2.xyz'].
        lines_before (int, optional): Number of lines to include before the keyword line. Defaults to 1.

    Returns:
        tuple: A tuple containing the lines around the keyword, the number of lines after the keyword, and the total charge.
    """
    if filenames is None:
        filenames = [file_path_1, file_path_2]
    data = []  # To store the lines around the keyword
    lines_after = 0  # Initialize lines_after to 0
    total_charge = None  # Initialize total_charge
    
    for filename in filenames:
        with open(filename, 'r') as file:
            previous_lines = []  # To store the lines before the keyword
            keyword_found = False
            for line in file:
                if keyword in line:
                    keyword_found = True
                    data.extend(previous_lines[-lines_before:])  # Add lines before the keyword
                    data.append(line.strip())  # Add the line containing the keyword
                    lines_after = int(previous_lines[-1].strip())
                    # Search for the charge value on the line containing the keyword
                    if ' q =' in line:
                        total_charge = line.split('q = ')[1].split()[0].strip()  # Extract the charge value
                    for _ in range(lines_after):
                        next_line = next(file, None)  # Move to the line after the keyword
                        if next_line:
                            data.append(next_line.strip())  # Add lines after the keyword
                    break  # Stop reading the file after finding the keyword
                previous_lines.append(line.strip())
        
        if keyword_found:
            break  # Stop searching in other files if keyword is found
    
    if not keyword_found:
        return "CSD code incorrect or not present in database", 0, None
    
    # Concatenate the lines into a single string
    return '\n'.join(data), lines_after, total_charge

**Example:**

In [ ]:
import coordchempy
from coordchempy import read_lines_around_keyword
lines, lines_after, total_charge = read_lines_around_keyword('KUMBAX')
print(lines)  # Output: XYZ data around 'KUMBAX'
print(lines_after)  # Output: Number of lines after the keyword
print(total_charge)  # Output: Total charge of the complex

**4. read_xyz**

Reads XYZ data around a keyword and returns atoms and coordinates.

In [ ]:
def read_xyz(keyword):  
    """
    Read XYZ data around a keyword.

    Args:
        keyword (str): The keyword to search for.

    Returns:
        tuple: A tuple containing lists of atoms and their coordinates.
    """
    # Call read_lines_around_keyword with default filenames to get the number of atom
    xyz_data, num_atoms_line, total_charge = read_lines_around_keyword(keyword)
    
    # Split the XYZ data into lines
    xyz_data_lines = xyz_data.split('\n')

    # Parse the XYZ data starting from the line after the CSD code
    atoms = []
    coordinates = []
    for line in xyz_data_lines[2:]:  # Skip the first two lines
        parts = line.split()
        if len(parts) == 4:  # Ensure the line contains atom data
            atoms.append(parts[0])
            coordinates.append((float(parts[1]), float(parts[2]), float(parts[3])))
    
    return atoms, coordinates

**Example:**

In [ ]:
import coordchempy
from coordchempy import read_xyz
atoms, coordinates = read_xyz('KUMBAX')
print(atoms)  # Output: List of atoms
print(coordinates)  # Output: List of coordinates
                    # Output: ['Ti', 'Cl', 'Cl', 'Cl', 'Si', 'Si', 'O', 'O', 'N', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H']
                    #[(9.9311675558673, 7.6389395163635, 14.06242636466981), (11.89023979210495, 8.26809372013789, 14.78070769423852), (10.69692689636514, 6.82343000073777, 12.20142095795944), (9.55873451491081, 5.72308113381907, 15.01791478924112), (9.18745152381335, 10.43497128550873, 13.64374044262446), (7.03097950471707, 8.23227786974568, 13.12728192739099), (9.25234511169237, 8.57811694985959, 16.15552678744744), (8.450252313592, 8.65374882645531, 10.68451538856048), (8.60945712626762, 8.80522489738401, 13.57756428776594), (9.85087491681459, 10.76068036339107, 15.41221909381012), (9.83870919713564, 9.80740991922912, 16.43682184293685), (10.3925124743748, 10.09666432651581, 17.67839437209156), (10.412536097802, 9.3594841816337, 18.46503923730392), (10.96117656051414, 11.33722591444766, 17.91502738832495), (11.39547361532039, 11.54450104302724, 18.88140230163064), (10.97579487791026, 12.29498861999595, 16.91968906531651), (11.41784729142005, 13.26297555206184, 17.09996252843956), (10.4269113965731, 11.99459916454058, 15.68575587411511), (10.44271249909679, 12.73806353568821, 14.90309322465675), (8.88630601479674, 7.78193179057208, 17.27677789361978), (9.76203721597875, 7.310820853995, 17.73613244226454), (8.22654926977827, 7.00014991893794, 16.89577620018798), (8.3524636083688, 8.38848596460022, 18.01547796084803), (7.86230930553524, 11.75613207407416, 13.27664820409585), (7.0205833851337, 11.68026452003058, 13.95585135033263), (7.47980596382085, 11.64790825644465, 12.2686276736438), (8.27569543278478, 12.75377215030569, 13.3723351710902), (10.67138251410527, 10.60365401154867, 12.4343482017134), (11.09231062485643, 9.62113243209324, 12.20796664297703), (11.46443114936509, 11.213497730445, 12.85897002696793), (10.36976537800108, 11.061038671201, 11.49691128419148), (5.85036373919955, 8.46269004470539, 14.61967029583179), (5.72591679833245, 9.51030157446582, 14.87358907868362), (6.24602722812008, 7.9550524164653, 15.49541133981302), (4.86895324727909, 8.04917781394758, 14.41433969765355), (7.16822998136751, 6.37097809722052, 12.70704306543974), (8.04798094304137, 6.17610398631367, 12.09436096713196), (6.29841545924022, 6.03142253907156, 12.15216342060753), (7.24021678932661, 5.76470237599724, 13.60886759041269), (6.39821561035415, 9.19794888520013, 11.62631572771209), (7.25949158868288, 9.28995548037147, 10.52485212240314), (6.87769191503973, 9.99060982784414, 9.38786381152245), (7.5314774013016, 10.06954951103202, 8.5332379196703), (5.63345148006191, 10.60042767242617, 9.34587710794095), (5.34139757112859, 11.14355256597967, 8.45897899065916), (4.77238433114735, 10.51595232562087, 10.42355702127492), (3.80390395381351, 10.99114129290503, 10.38333181597284), (5.16224799383735, 9.81621832382147, 11.55479762025347), (4.49349146106097, 9.7471411907259, 12.39865154498646), (9.3468139897706, 8.61193797256253, 9.59704706790516), (8.88929509647994, 8.13508117851874, 8.72415344699606), (10.19006791173813, 8.0134616678729, 9.93753460896899), (9.69392238084811, 9.61392806213496, 9.32462911770203)]

**5. infer_bonds**

Infers bonds between atoms based on distances and covalent radii. A limitation to this function is that it takes a lot of time to run when faced with a large molecules. 

In [ ]:
def infer_bonds(atoms, coordinates, tolerance=0.4,):
    """
    Infer bonds between atoms based on distances and covalent radii.

    Args:
        atoms (list): List of atom symbols.
        coordinates (list): List of atom coordinates.
        tolerance (float, optional): Tolerance factor for bond determination. Defaults to 0.4.

    Returns:
        list: List of tuples representing bonded atom indices.
    """
    bonds = []  # Initialize an empty list to store the bonds.
    
    num_atoms = len(atoms)  # Get the total number of atoms in the molecule.
    
    # Iterate over each pair of atoms to check for potential bonds.
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            atom1, atom2 = atoms[i], atoms[j]  # Get the symbols of the two atoms being considered.
            coord1, coord2 = coordinates[i], coordinates[j]  # Get the 3D coordinates of these atoms.
            
            distance = calculate_distance(coord1, coord2)  # Calculate the Euclidean distance between the two atoms.
            
            # Calculate the maximum allowable distance for a bond, which is the sum of their covalent radii plus a tolerance.
            max_distance = get_covalent_radius(atom1) + get_covalent_radius(atom2) + tolerance
            
            # If the actual distance is less than or equal to the maximum allowable distance, a bond is inferred.
            if distance <= max_distance:
                bonds.append((i, j))  # Add the indices of the bonded atoms as a tuple to the bonds list.
    
    return bonds  # Return the list of inferred bonds.

**Example:**

In [ ]:
import coordchempy
from coordchempy import infer_bonds
coordinates = [(9.93116755586730, 7.63893951636350, 14.06242636466981),
            (11.89023979210495, 8.26809372013789, 14.78070769423852),
            (10.69692689636514, 6.82343000073777, 12.20142095795944),
            (9.55873451491081, 5.72308113381907, 15.01791478924112),
            (9.18745152381335, 10.43497128550873, 13.64374044262446),
            (7.03097950471707, 8.23227786974568, 13.12728192739099),
            (9.25234511169237, 8.57811694985959, 16.15552678744744),
            (8.45025231359200, 8.65374882645531, 10.68451538856048),
            (8.60945712626762, 8.80522489738401, 13.57756428776594),
            (9.85087491681459, 10.76068036339107, 15.41221909381012),
            (9.83870919713564, 9.80740991922912, 16.43682184293685),
            (10.39251247437480, 10.09666432651581, 17.67839437209156),
            (10.41253609780200, 9.35948418163370, 18.46503923730392),
            (10.96117656051414, 11.33722591444766, 17.91502738832495),
            (11.39547361532039, 11.54450104302724, 18.88140230163064),
            (10.97579487791026, 12.29498861999595, 16.91968906531651),
            (11.41784729142005, 13.26297555206184, 17.09996252843956),
            (10.42691139657310, 11.99459916454058, 15.68575587411511),
            (10.44271249909679, 12.73806353568821, 14.90309322465675),
            (8.88630601479674, 7.78193179057208, 17.27677789361978),
            (9.76203721597875, 7.31082085399500, 17.73613244226454),
            (8.22654926977827, 7.00014991893794, 16.89577620018798),
            (8.35246360836880, 8.38848596460022, 18.01547796084803),
            (7.86230930553524, 11.75613207407416, 13.27664820409585),
            (7.02058338513370, 11.68026452003058, 13.95585135033263),
            (7.47980596382085, 11.64790825644465, 12.26862767364380),
            (8.27569543278478, 12.75377215030569, 13.37233517109020),
            (10.67138251410527, 10.60365401154867, 12.43434820171340),
            (11.09231062485643, 9.62113243209324, 12.20796664297703),
            (11.46443114936509, 11.21349773044500, 12.85897002696793),
            (10.36976537800108, 11.06103867120100, 11.49691128419148),
            (5.85036373919955, 8.46269004470539, 14.61967029583179),
            (5.72591679833245, 9.51030157446582, 14.87358907868362),
            (6.24602722812008, 7.95505241646530, 15.49541133981302),
            (4.86895324727909, 8.04917781394758, 14.41433969765355),
            (7.16822998136751, 6.37097809722052, 12.70704306543974),
            (8.04798094304137, 6.17610398631367, 12.09436096713196),
            (6.29841545924022, 6.03142253907156, 12.15216342060753),
            (7.24021678932661, 5.76470237599724, 13.60886759041269),
            (6.39821561035415, 9.19794888520013, 11.62631572771209),
            (7.25949158868288, 9.28995548037147, 10.52485212240314),
            (6.87769191503973, 9.99060982784414, 9.38786381152245),
            (7.53147740130160, 10.06954951103202, 8.53323791967030),
            (5.63345148006191, 10.60042767242617, 9.34587710794095),
            (5.34139757112859, 11.14355256597967, 8.45897899065916),
            (4.77238433114735, 10.51595232562087, 10.42355702127492),
            (3.80390395381351, 10.99114129290503, 10.38333181597284),
            (5.16224799383735, 9.81621832382147, 11.55479762025347),
            (4.49349146106097, 9.74714119072590, 12.39865154498646),
            (9.34681398977060, 8.61193797256253, 9.59704706790516),
            (8.88929509647994, 8.13508117851874, 8.72415344699606),
            (10.19006791173813, 8.01346166787290, 9.93753460896899),
            (9.69392238084811, 9.61392806213496, 9.32462911770203)]
atoms = ['Ti', 'Cl', 'Cl', 'Cl', 'Si', 'Si', 'O', 'O', 'N', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C',
            'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'C',
            'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H']
bonds = infer_bonds(atoms, coordinates)
print(bonds)  # Output: List of bonded atom indices
              #  Output: [(0, 1), (0, 2), (0, 3), (0, 8), (4, 8), (4, 9), (4, 23), (4, 27), (5, 8), (5, 31), (5, 35), (5, 39), (6, 10), (6, 19), (7, 40), (7, 49), (9, 10), (9, 17), (10, 11), (11, 12), (11, 13), (13, 14), (13, 15), (15, 16), (15, 17), (17, 18), (19, 20), (19, 21), (19, 22), (23, 24), (23, 25), (23, 26), (27, 28), (27, 29), (27, 30), (31, 32), (31, 33), (31, 34), (35, 36), (35, 37), (35, 38), (39, 40), (39, 47), (40, 41), (41, 42), (41, 43), (43, 44), (43, 45), (45, 46), (45, 47), (47, 48), (49, 50), (49, 51), (49, 52)]

**6. find_central_atom**

Finds the central atom in a molecule based on transition metal criteria.

In [ ]:
def find_central_atom(atoms):
    """
    Find the central atom in a molecule based on transition metal criteria.

    Args:
        atoms (list): List of atom symbols.

    Returns:
        tuple: A tuple containing the symbol and index of the central atom.
    """
    # Define a set of atomic numbers corresponding to transition metals.
    transition_metal_atomic_numbers = set(range(21, 31)) | set(range(39, 49)) | set(range(72, 81)) | set(range(104, 113)) | {57, 89}
    
    # Iterate over the list of atoms, with 'i' as the index and 'atom' as the atom symbol.
    for i, atom in enumerate(atoms):
        # Check if the atomic number of the current atom is in the set of transition metal atomic numbers.
        if element(atom).atomic_number in transition_metal_atomic_numbers:
            return atom, i  # Return the atom symbol and its index if it's a transition metal.
    
    # If no transition metal is found, raise a ValueError.
    raise ValueError("No central atom found in the molecule.")

**Examples:**

In [ ]:
import coordchempy
from coordchempy import find_central_atom
atoms = ['Ti', 'Cl', 'Cl', 'Cl', 'Si', 'Si', 'O', 'O', 'N', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C',
            'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'C',
            'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H']
central_atom, index = find_central_atom(atoms)
print(central_atom)  # Output: Symbol of the central atom
print(index)  # Output: Index of the central atom

**7. find_ligands**

Identifies the atoms directly linked to the central atom. For certain compounds, it gives the wrong number of Ligands (ex: RUNCIM)    

In [ ]:
def find_ligands(atoms, coordinates, tolerance=0.4):
    """
    Identify the atoms directly linked to the central atom.

    Args:
        atoms (list): List of atom symbols.
        coordinates (list): List of atom coordinates.
        tolerance (float, optional): Tolerance factor for bond determination. Defaults to 0.4.

    Returns:
        list: List of the atom symbols linked to the central one.
    """
    # Find the central atom and its index in the list of atoms.
    _, central_atom_index = find_central_atom(atoms)
    
    # Infer bonds between atoms based on distances and covalent radii.
    bonds = infer_bonds(atoms, coordinates, tolerance)
    
    # Initialize an empty list to store the symbols of the ligands.
    ligands = []
    
    # Iterate over the list of inferred bonds.
    for bond in bonds:
        # Check if the central atom index is part of the current bond.
        if central_atom_index in bond:
            # Determine the index of the ligand atom in the bond.
            ligand_index = bond[1] if bond[0] == central_atom_index else bond[0]
            # Append the symbol of the ligand atom to the ligands list.
            ligands.append(atoms[ligand_index])
    
    # Return the list of ligand atom symbols.
    return ligands

**Example:**

In [ ]:
import coordchempy
from coordchempy import find_ligands
coordinates = [(9.93116755586730, 7.63893951636350, 14.06242636466981),
            (11.89023979210495, 8.26809372013789, 14.78070769423852),
            (10.69692689636514, 6.82343000073777, 12.20142095795944),
            (9.55873451491081, 5.72308113381907, 15.01791478924112),
            (9.18745152381335, 10.43497128550873, 13.64374044262446),
            (7.03097950471707, 8.23227786974568, 13.12728192739099),
            (9.25234511169237, 8.57811694985959, 16.15552678744744),
            (8.45025231359200, 8.65374882645531, 10.68451538856048),
            (8.60945712626762, 8.80522489738401, 13.57756428776594),
            (9.85087491681459, 10.76068036339107, 15.41221909381012),
            (9.83870919713564, 9.80740991922912, 16.43682184293685),
            (10.39251247437480, 10.09666432651581, 17.67839437209156),
            (10.41253609780200, 9.35948418163370, 18.46503923730392),
            (10.96117656051414, 11.33722591444766, 17.91502738832495),
            (11.39547361532039, 11.54450104302724, 18.88140230163064),
            (10.97579487791026, 12.29498861999595, 16.91968906531651),
            (11.41784729142005, 13.26297555206184, 17.09996252843956),
            (10.42691139657310, 11.99459916454058, 15.68575587411511),
            (10.44271249909679, 12.73806353568821, 14.90309322465675),
            (8.88630601479674, 7.78193179057208, 17.27677789361978),
            (9.76203721597875, 7.31082085399500, 17.73613244226454),
            (8.22654926977827, 7.00014991893794, 16.89577620018798),
            (8.35246360836880, 8.38848596460022, 18.01547796084803),
            (7.86230930553524, 11.75613207407416, 13.27664820409585),
            (7.02058338513370, 11.68026452003058, 13.95585135033263),
            (7.47980596382085, 11.64790825644465, 12.26862767364380),
            (8.27569543278478, 12.75377215030569, 13.37233517109020),
            (10.67138251410527, 10.60365401154867, 12.43434820171340),
            (11.09231062485643, 9.62113243209324, 12.20796664297703),
            (11.46443114936509, 11.21349773044500, 12.85897002696793),
            (10.36976537800108, 11.06103867120100, 11.49691128419148),
            (5.85036373919955, 8.46269004470539, 14.61967029583179),
            (5.72591679833245, 9.51030157446582, 14.87358907868362),
            (6.24602722812008, 7.95505241646530, 15.49541133981302),
            (4.86895324727909, 8.04917781394758, 14.41433969765355),
            (7.16822998136751, 6.37097809722052, 12.70704306543974),
            (8.04798094304137, 6.17610398631367, 12.09436096713196),
            (6.29841545924022, 6.03142253907156, 12.15216342060753),
            (7.24021678932661, 5.76470237599724, 13.60886759041269),
            (6.39821561035415, 9.19794888520013, 11.62631572771209),
            (7.25949158868288, 9.28995548037147, 10.52485212240314),
            (6.87769191503973, 9.99060982784414, 9.38786381152245),
            (7.53147740130160, 10.06954951103202, 8.53323791967030),
            (5.63345148006191, 10.60042767242617, 9.34587710794095),
            (5.34139757112859, 11.14355256597967, 8.45897899065916),
            (4.77238433114735, 10.51595232562087, 10.42355702127492),
            (3.80390395381351, 10.99114129290503, 10.38333181597284),
            (5.16224799383735, 9.81621832382147, 11.55479762025347),
            (4.49349146106097, 9.74714119072590, 12.39865154498646),
            (9.34681398977060, 8.61193797256253, 9.59704706790516),
            (8.88929509647994, 8.13508117851874, 8.72415344699606),
            (10.19006791173813, 8.01346166787290, 9.93753460896899),
            (9.69392238084811, 9.61392806213496, 9.32462911770203)]
atoms = ['Ti', 'Cl', 'Cl', 'Cl', 'Si', 'Si', 'O', 'O', 'N', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C',
            'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'C',
            'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H']
ligands = find_ligands(atoms, coordinates)
print(ligands)  # Output: List of ligand atom symbols

**8. find_actual_ligand_count**

Finds the actual count of ligands bonded to the central atom. or certain compounds, it gives the wrong number of Ligands (ex: RUNCIM) 

In [ ]:

def find_actual_ligand_count(atoms, bonds, central_atom_index):
    """
    Find the actual count of ligands bonded to the central atom.

    Args:
        atoms (list): List of atom symbols.
        coordinates (list): List of atom coordinates.
        bonds (list): List of bonded atom indices.
        central_atom_index (int): Index of the central atom.

    Returns:
        int: The count of unique ligands.
    """
    # Create a graph adjacency list
    graph = {i: [] for i in range(len(atoms))}
    for bond in bonds:
        atom1_index, atom2_index = bond
        graph[atom1_index].append(atom2_index)
        graph[atom2_index].append(atom1_index)

    # Identify atoms directly bonded to the central atom
    directly_bonded = [bond[1] if bond[0] == central_atom_index else bond[0] for bond in bonds if central_atom_index in bond]

    # Function to perform DFS and identify connected ligand atoms without go through the same ligand twice.
    def dfs(node, visited):
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited and neighbor != central_atom_index:
                dfs(neighbor, visited)

    # Find unique ligands using DFS
    unique_ligands = []
    visited = set()
    for atom in directly_bonded:
        if atom not in visited:
            ligand_group = set()
            dfs(atom, ligand_group)
            unique_ligands.append(ligand_group)
            visited |= ligand_group

    return len(unique_ligands)

**Example:**

In [ ]:
import coordchempy
from coordchempy import find_actual_ligand_count
atoms = ['Ti', 'Cl', 'Cl', 'Cl', 'Si', 'Si', 'O', 'O', 'N', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C',
         'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'C',
         'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H']
bonds = [(0, 1), (0, 2), (0, 3), (0, 8), (4, 8), (4, 9), (4, 23), (4, 27), (5, 8), (5, 31), (5, 35), (5, 39), (6, 10), 
         (6, 19), (7, 40), (7, 49), (9, 10), (9, 17), (10, 11), (11, 12), (11, 13), (13, 14), (13, 15), (15, 16), 
         (15, 17), (17, 18), (19, 20), (19, 21), (19, 22), (23, 24), (23, 25), (23, 26), (27, 28), (27, 29), (27, 30), 
         (31, 32), (31, 33), (31, 34), (35, 36), (35, 37), (35, 38), (39, 40), (39, 47), (40, 41), (41, 42), (41, 43), 
         (43, 44), (43, 45), (45, 46), (45, 47), (47, 48), (49, 50), (49, 51), (49, 52)]
central_atom_index = 0
ligand_count = find_actual_ligand_count(atoms, bonds, central_atom_index)
print(ligand_count)  # Output: Number of unique ligands 

**9. calculate_angle**

Calculates the angle between three points in 3D space.

In [ ]:
def calculate_angle(coord1, coord2, coord3):
    """
    Calculate the angle between three points in 3D space.

    Args:
        coord1 (tuple): Coordinates of the first point.
        coord2 (tuple): Coordinates of the second point.
        coord3 (tuple): Coordinates of the third point.

    Returns:
        float: The angle in degrees.
    """
    # Convert coordinates to numpy arrays
    v1 = np.array(coord1) - np.array(coord2)
    v2 = np.array(coord3) - np.array(coord2)
    
    # Calculate the dot product and magnitudes of the vectors
    dot_product = np.dot(v1, v2)
    mag_v1 = np.linalg.norm(v1)
    mag_v2 = np.linalg.norm(v2)
    
    # Calculate the cosine of the angle
    cos_theta = dot_product / (mag_v1 * mag_v2)
    
    # Calculate the angle in radians and then convert to degrees
    angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

**Example:**

In [ ]:
import coordchempy
from coordchempy import calculate_angle
angle = calculate_angle((0, 0, 0), (1, 1, 1), (2, 2, 2))
print(angle)  # Output: Angle in degrees

**10. determine_geometry**

Determines the molecular geometry based on bond angles and coordination number. Due to the angles being too different from the angles for the ordinary geometry, margines are used in order to find some geometries.    

In [ ]:
def determine_geometry(angles, coordination_number):
    """
    Determine the molecular geometry based on bond angles and coordination number.

    Args:
        angles (list): List of bond angles.
        coordination_number (int): The coordination number of the central atom.

    Returns:
        str: The name of the molecular geometry.
    """
    # Check if the coordination number is 2.
    if coordination_number == 2:
        # For coordination number 2, if there is exactly one angle between 160 and 180 degrees, it's linear.
        if len(angles) == 1 and 160 <= angles[0] <= 180:
            return 'linear'

    # Check if the coordination number is 3.
    if coordination_number == 3:
        # For coordination number 3, if there are three angles between 95 and 145 degrees, it's trigonal planar.
        if len(angles) == 3 and all(95 <= angle <= 145 for angle in angles):
            return 'trigonal planar'

    # Check if the coordination number is 4.
    if coordination_number == 4:
        # For coordination number 4, if there are six angles, it could be square planar or tetrahedral.
        if len(angles) == 6:
            # If two angles are between 160 and 180 degrees and the rest are between 65 and 115 degrees, it's square planar.
            if sum(160 <= angle <= 180 for angle in angles) == 2 and all(65 <= angle <= 115 for angle in angles if not (160 <= angle <= 180)):
                return 'square planar'
            # If all angles are between 85 and 135 degrees, it's tetrahedral.
            elif all(85 <= angle <= 135 for angle in angles):
                return 'tetrahedral'

    # Check if the coordination number is 5.
    if coordination_number == 5:
        # For coordination number 5, if there are ten angles, it could be trigonal bipyramidal, square pyramidal, or seesaw.
        if len(angles) == 10:
            # If one angle is between 160 and 180 degrees, three angles are between 95 and 145 degrees, and the rest are between 65 and 115 degrees, it's trigonal bipyramidal.
            if sum(160 <= angle <= 180 for angle in angles) == 1 and sum(95 <= angle <= 145 for angle in angles) == 3 and all(65 <= angle <= 115 for angle in angles if not (95 <= angle <= 145 or 160 <= angle <= 180)):
                return 'trigonal bipyramidal'
            # If any angle is between 55 and 95 degrees and the rest are between 75 and 115 degrees, it's square pyramidal.
            elif any(55 <= angle <= 95 for angle in angles) and all(75 <= angle <= 115 for angle in angles if angle > 95):
                return 'square pyramidal'
            # If there are three distinct angle values, any angle is between 65 and 95 degrees, and the maximum angle is between 105 and 135 degrees, it's seesaw.
            elif len(set([round(angle) for angle in angles])) == 3 and any(65 <= angle <= 95 for angle in angles) and 105 <= max(angles) <= 135:
                return 'seesaw'

    # Check if the coordination number is 6.
    if coordination_number == 6:
        # For coordination number 6, if there are fifteen angles, it could be octahedral.
        if len(angles) == 15:
            # If three angles are between 160 and 180 degrees and the rest are between 65 and 115 degrees, it's octahedral.
            if sum(160 <= angle <= 180 for angle in angles) == 3 and all(65 <= angle <= 115 for angle in angles if not (160 <= angle <= 180)):
                return 'octahedral'

    # If no known geometry matches, return 'unknown'.
    return 'unknown'

**Example:**

In [ ]:
import coordchempy
from coordchempy import determine_geometry
geometry = determine_geometry([90, 170, 175, 70, 100, 80], 4)
print(geometry)  # Output: Name of the molecular geometry

**11. calculate_angles_and_geometry** 

Calculates bond angles and determines molecular geometry.

In [ ]:
def calculate_angles_and_geometry(atoms, coordinates):
    """
    Calculate bond angles and determine molecular geometry.

    Args:
        atoms (list): List of atom symbols.
        coordinates (list): List of atom coordinates.

    Returns:
        tuple: A tuple containing lists of bond angles and the name of the molecular geometry.
    """
    # Find the central atom and its index in the molecule.
    _, central_atom_index = find_central_atom(atoms)

    # Infer the bonds between atoms based on their coordinates and covalent radii.
    bonds = infer_bonds(atoms, coordinates)

    # Identify the indices of the ligands directly bonded to the central atom.
    ligand_indices = [i for i, atom in enumerate(atoms) if (central_atom_index, i) in bonds or (i, central_atom_index) in bonds]

    # Initialize an empty list to store the bond angles.
    angles = []

    # Calculate bond angles between each pair of ligands.
    for i in range(len(ligand_indices)):
        for j in range(i + 1, len(ligand_indices)):
            # Calculate the bond angle between ligand i, the central atom, and ligand j.
            angle = calculate_angle(coordinates[ligand_indices[i]], coordinates[central_atom_index], coordinates[ligand_indices[j]])
            # Append the calculated angle to the angles list.
            angles.append(angle)

    # Determine the coordination number based on the number of ligands.
    coordination_number = len(ligand_indices)

    # Determine the molecular geometry based on the calculated bond angles and coordination number.
    geometry = determine_geometry(angles, coordination_number)

    # Return the list of bond angles and the name of the molecular geometry.
    return angles, geometry

**Examples:**

In [ ]:
import coordchempy
from coordchempy import calculate_angles_and_geometry
coordinates = [(9.93116755586730, 7.63893951636350, 14.06242636466981),
            (11.89023979210495, 8.26809372013789, 14.78070769423852),
            (10.69692689636514, 6.82343000073777, 12.20142095795944),
            (9.55873451491081, 5.72308113381907, 15.01791478924112),
            (9.18745152381335, 10.43497128550873, 13.64374044262446),
            (7.03097950471707, 8.23227786974568, 13.12728192739099),
            (9.25234511169237, 8.57811694985959, 16.15552678744744),
            (8.45025231359200, 8.65374882645531, 10.68451538856048),
            (8.60945712626762, 8.80522489738401, 13.57756428776594),
            (9.85087491681459, 10.76068036339107, 15.41221909381012),
            (9.83870919713564, 9.80740991922912, 16.43682184293685),
            (10.39251247437480, 10.09666432651581, 17.67839437209156),
            (10.41253609780200, 9.35948418163370, 18.46503923730392),
            (10.96117656051414, 11.33722591444766, 17.91502738832495),
            (11.39547361532039, 11.54450104302724, 18.88140230163064),
            (10.97579487791026, 12.29498861999595, 16.91968906531651),
            (11.41784729142005, 13.26297555206184, 17.09996252843956),
            (10.42691139657310, 11.99459916454058, 15.68575587411511),
            (10.44271249909679, 12.73806353568821, 14.90309322465675),
            (8.88630601479674, 7.78193179057208, 17.27677789361978),
            (9.76203721597875, 7.31082085399500, 17.73613244226454),
            (8.22654926977827, 7.00014991893794, 16.89577620018798),
            (8.35246360836880, 8.38848596460022, 18.01547796084803),
            (7.86230930553524, 11.75613207407416, 13.27664820409585),
            (7.02058338513370, 11.68026452003058, 13.95585135033263),
            (7.47980596382085, 11.64790825644465, 12.26862767364380),
            (8.27569543278478, 12.75377215030569, 13.37233517109020),
            (10.67138251410527, 10.60365401154867, 12.43434820171340),
            (11.09231062485643, 9.62113243209324, 12.20796664297703),
            (11.46443114936509, 11.21349773044500, 12.85897002696793),
            (10.36976537800108, 11.06103867120100, 11.49691128419148),
            (5.85036373919955, 8.46269004470539, 14.61967029583179),
            (5.72591679833245, 9.51030157446582, 14.87358907868362),
            (6.24602722812008, 7.95505241646530, 15.49541133981302),
            (4.86895324727909, 8.04917781394758, 14.41433969765355),
            (7.16822998136751, 6.37097809722052, 12.70704306543974),
            (8.04798094304137, 6.17610398631367, 12.09436096713196),
            (6.29841545924022, 6.03142253907156, 12.15216342060753),
            (7.24021678932661, 5.76470237599724, 13.60886759041269),
            (6.39821561035415, 9.19794888520013, 11.62631572771209),
            (7.25949158868288, 9.28995548037147, 10.52485212240314),
            (6.87769191503973, 9.99060982784414, 9.38786381152245),
            (7.53147740130160, 10.06954951103202, 8.53323791967030),
            (5.63345148006191, 10.60042767242617, 9.34587710794095),
            (5.34139757112859, 11.14355256597967, 8.45897899065916),
            (4.77238433114735, 10.51595232562087, 10.42355702127492),
            (3.80390395381351, 10.99114129290503, 10.38333181597284),
            (5.16224799383735, 9.81621832382147, 11.55479762025347),
            (4.49349146106097, 9.74714119072590, 12.39865154498646),
            (9.34681398977060, 8.61193797256253, 9.59704706790516),
            (8.88929509647994, 8.13508117851874, 8.72415344699606),
            (10.19006791173813, 8.01346166787290, 9.93753460896899),
            (9.69392238084811, 9.61392806213496, 9.32462911770203)]
atoms = ['Ti', 'Cl', 'Cl', 'Cl', 'Si', 'Si', 'O', 'O', 'N', 'C', 'C', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C',
            'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'H', 'H', 'H', 'C', 'C',
            'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'C', 'H', 'H', 'H']

angles, geometry = calculate_angles_and_geometry(atoms, coordinates)
print(angles)  # Output: List of bond angles
print(geometry)  # Output: Name of the molecular geometry

**12. cn**

Returns the coordination number.

In [ ]:
def cn(keyword):
    """
    Return the coordination number

    Args:
        keyword (str): The keyword used to identify the molecule.
    
    Returns:
        int : the coordination number
    """
    atoms, coordinates = read_xyz(keyword)
    cn = len(find_ligands(atoms, coordinates))
    return cn

**Examples:**

In [ ]:
import coordchempy
from coordchempy import cn
coordination_number = cn('KUMBAX')
print(coordination_number)  # Output: Coordination number

**13. charge**

Returns the total charge of the coordinate compound.

In [ ]:
def charge(keyword):
    """
    Return the total charge of the coordinate compound

    Args:
        keyword (str): The keyword used to identify the molecule.
    
    Returns:
        int : the charge of the coordinate compound
    """
    _, _, total_charge = read_lines_around_keyword(keyword)
    return int(total_charge)

**Example:**

In [ ]:
import coordchempy
from coordchempy import charge
total_charge = charge('KUMBAX')
print(total_charge)  # Output: Total charge

**14. visualize_label**

Visualizes the molecule with labeled atoms.

In [ ]:
def visualize_label(keyword):
    """
    Visualize the molecule specified by the keyword with the label of the atoms.

    Args:
        keyword (str): The keyword used to identify the molecule.
    """
    atoms, _ = read_xyz(keyword)
    xyz_data, _, _ = read_lines_around_keyword(keyword)

    # Initialize a viewer
    viewer = py3Dmol.view()

    # Add a model from the .xyz data
    viewer.addModel(xyz_data, 'xyz')

    # Set the style
    viewer.setStyle({'stick': {}})

    # Label each atom
    for i, atom in enumerate(atoms):
        coord = coordinates[i]
        label = f"{atom}"
        viewer.addLabel(label, {'position': {'x': coord[0], 'y': coord[1], 'z': coord[2]},
                'backgroundColor': 'rgba(255, 255, 255, 0.5)',  # Light background
                'fontColor': 'black',
                'fontSize': 10,  # Smaller font size
                'padding': 0,
                'borderThickness': 0})

    # Center the molecule
    viewer.zoomTo()

    # Render the viewer
    viewer.show()

**Examples:**

In [ ]:
import coordchempy
from coordchempy import visualize_label
visualize_label('KOFHUL')  # Displays the 3D visualization with labeled atoms

**15. visualize**

Visualizes the molecule. For certain compounds, py3Dmol sometimes bonds the wrong atoms or misses bonds. Also the double and triple bond are not displayed.  

In [ ]:
def visualize (keyword):
    """
    Visualize the molecule specified by the keyword.

    Args:
        keyword (str): The keyword used to identify the molecule.
    """
    atoms, _ = read_xyz(keyword)
    xyz_data, _, _ = read_lines_around_keyword(keyword)

    # Initialize a viewer
    viewer = py3Dmol.view()

    # Add a model from the .xyz data
    viewer.addModel(xyz_data, 'xyz')

    # Set the style
    viewer.setStyle({'stick': {}})

    # Center the molecule
    viewer.zoomTo()

    # Render the viewer
    viewer.show()

**Example:**

In [ ]:
import coordchempy
from coordchempy import visualize
visualize('KUMBAX')  # Displays the 3D visualization with a suplementary wrong bond on the transition metal
visualize('KOFHUL')  # Displays the true 3D visualization

**16. visualize_all_data**

Visualizes the molecule and prints its characteristics.

In [ ]:
def visualize_all_data(keyword):
    """
    Visualize the molecule specified by the keyword and print its characteristics.

    Args:
        keyword (str): The keyword used to identify the molecule.
    """
    atoms, coordinates = read_xyz(keyword)
    cn = len(find_ligands(atoms, coordinates))
    bonds = infer_bonds(atoms, coordinates)
    central_atom_symbol, central_atom_index = find_central_atom(atoms)
    ligand_counts = find_actual_ligand_count(atoms, coordinates, bonds, central_atom_index)
    angles, geometry = calculate_angles_and_geometry(atoms, coordinates)
    xyz_data, lines_after, total_charge = read_lines_around_keyword(keyword)
    
    # Print the symbol of the central atom
    print(f"The symbol of the central atom is {central_atom_symbol}.")

    # Print the total charge if available
    print(f"The total charge of the complex is {total_charge}.")

    # Print the coordination number and the number of ligands
    if cn != ligand_counts:
        print(f"The coordination number of the transition metal is {cn} and the number of ligands is {ligand_counts}.")
    if cn == ligand_counts:
        print(f"The coordination number of the transition metal and the number of ligands is {cn}.")

    # Print the angles and geometry
    print(f"The angles measure {angles} and the geometry of the molecule is {geometry}.")

    # Initialize a viewer
    viewer = py3Dmol.view()

    # Add a model from the .xyz data
    viewer.addModel(xyz_data, 'xyz')

    # Set the style
    viewer.setStyle({'stick': {}})

    # Label each atom
    for i, atom in enumerate(atoms):
        coord = coordinates[i]
        label = f"{atom}"
        viewer.addLabel(label, {'position': {'x': coord[0], 'y': coord[1], 'z': coord[2]},
                'backgroundColor': 'rgba(255, 255, 255, 0.5)',  # Light background
                'fontColor': 'black',
                'fontSize': 10,  # Smaller font size
                'padding': 0,
                'borderThickness': 0})

    # Center the molecule
    viewer.zoomTo()

    # Render the viewer
    viewer.show()

**Example:**

In [ ]:
import coordchempy
from coordchempy import visualize_all_data
visualize_all_data('KOFHUL')  # Displays the 3D visualization and prints the molecule's characteristics

## Difficulties encountered
<div style="text-align: justify;">
The first challenge of our project was to find a good package in order to model the coordination compound based on an xyz file. We choose to use py3Dmol which work well most of the time but sometime the bonds are not well modelized by py3Dmol. We tried to fix this by converting the xyz file into a pdb file which allows to specify the bond but we could not make it work correctly.

Another problem is the time of execution for some function which could take from one to thirty minutes depending on the size of the molecule. This issue was constraining in the way that it greatly increased the time for the accomplishment of simple task such as the tests for our functions. This problem could have been solved with a better optimization of certain functions.

Moreover, the function determine_geometry was a problem since none of the compounds had the exact angle of the given geometries and a lot of the coordination compounds did not have a common geometry. To overcome this, large margins have been used which results to possible error when the geometries are determined.


## Conclusion
<div style="text-align: justify;">
In conclusion, this program serves as a robust toolkit for the analysis and visualization of coordination compounds. By integrating a suite of functions tailored to read, process, and interpret molecular data, infer bonds, and identify key molecular components such as central atoms and ligands, this software package offers a comprehensive solution for researchers and practitioners in the field of coordination chemistry.

Through the seamless combination of various functionalities, users gain access to valuable insights into the geometric arrangements and properties of complex molecules, enabling deeper understanding and exploration of molecular structures. The examples provided throughout this report highlight the versatility and effectiveness of each function within the toolkit, demonstrating their utility in achieving the project's overarching objectives.

From reading molecular data  to determining molecular geometry and visualizing structures, each function plays a crucial role in facilitating molecular analysis and interpretation. Moreover, the visualization capabilities provided by the program offer users an intuitive means of exploring molecular structures in a three-dimensional space, enhancing comprehension and facilitating communication of complex structural information.
In summary, this program provids researchers and students with a powerful tool for the analysis and visualization of coordination compounds. 

Finally, due to informatic issues Michael, was not able to commit anything to the repository. It was, therefore discussed with the professor that it would be convenient for the group if Mateo commit all elements in the repository.